The first cell below must be run to do all the setup for what comes next.

In [59]:
import docker
import kubernetes
import requests
import os
import subprocess
import time

session = requests.Session()
session.headers.update({'x-tyk-authorization': 'stuff'})

tyk_namespace = 'tyk'
service_namespace = 'http-jokes'

def cleanup_apis(requests_session):
    for api in requests_session.get("http://localhost:8080/tyk/apis/").json():
        print(api['api_id'])
        print(requests_session.delete("http://localhost:8080/tyk/apis/" + api['api_id']).text)
    requests_session.get('http://localhost:8080/tyk/reload/group').text

def define_api(openapi_file, requests_session):
    with open(openapi_file, 'r') as f:
        api_definition = f.read()
    response = requests_session.post("http://localhost:8080/tyk/apis/oas/import", data=api_definition)
    print(response.json())
    requests_session.get('http://localhost:8080/tyk/reload/group').text

def build_api_image():
    version = '1.0.0' # TODO: read from pyproject.toml
    client = docker.from_env()
    image, _ = client.images.build(path='.', tag=f'http-jokes:{version}', rm=True)
    image.tag(f'localhost:5001/http-jokes:{version}')
    client.images.push(f'localhost:5001/http-jokes:{version}')

def install_tyk():
    client, config = kubernetes.client, kubernetes.config
    config.load_kube_config()
    k8s = client.CoreV1Api()

    create_k8s_namespace(client, k8s, tyk_namespace)

    try:
        k8s.read_namespaced_config_map('tyk-secret', namespace=tyk_namespace)
    except kubernetes.client.rest.ApiException:
        cmap = client.V1ConfigMap()
        cmap.metadata = client.V1ObjectMeta(name='tyk-secret')
        cmap.data = {'APISecret': 'stuff'}
        k8s.create_namespaced_config_map(namespace=tyk_namespace, body=cmap)

    os.system(f'helm upgrade tyk-redis oci://registry-1.docker.io/bitnamicharts/redis -n {tyk_namespace} --install --version 19.0.2 --set replica.replicaCount=1 --wait')
    os.system(f'helm upgrade tyk-oss tyk-helm/tyk-oss -n {tyk_namespace} --install -f tyk-values.yaml --wait')

def create_k8s_namespace(client, k8s, namespace):
    try:
        ns = k8s.read_namespace(namespace)
    except kubernetes.client.rest.ApiException:
        # create namespace
        ns = client.V1Namespace(metadata=client.V1ObjectMeta(name=namespace))
        k8s.create_namespace(ns)
    return ns

def kill_namespace(client, k8s, namespace):
    try:
        k8s.delete_namespace(namespace, body=client.V1DeleteOptions(name=namespace), grace_period_seconds=0, force=True)
    except kubernetes.client.rest.ApiException:
        pass

The process starts with installing and running Tyk. Uncomment the `install_tyk()` line if you do not have it running. This uses the 'hello' endpoint in Tyk to make sure that it is running. If you do not see a line like the following, then something is not configured properly.

```
{'status': 'pass', 'version': '5.5.0', 'description': 'Tyk GW', 'details': {'redis': {'status': 'pass', 'componentType': 'datastore', 'time': '2024-09-26T19:57:37Z'}}}
```

In [63]:
# if needed, run this cell to get tyk up and running
#install_tyk()
port_fwd = subprocess.Popen(['kubectl', '-n', tyk_namespace, 'port-forward', 'svc/gateway-svc-tyk-oss-tyk-gateway', '8080:8080'])
time.sleep(1)
print(requests.get('http://localhost:8080/hello').json())

Forwarding from 127.0.0.1:8080 -> 8080
Forwarding from [::1]:8080 -> 8080
Handling connection for 8080
{'status': 'pass', 'version': '5.5.0', 'description': 'Tyk GW', 'details': {'redis': {'status': 'pass', 'componentType': 'datastore', 'time': '2024-09-26T20:46:37Z'}}}


The next block builds a docker image of the Jokes service code in `api/main.py` and then pushes it to a local registry. It then deploys that container to your k8s instance and registers the service with Tyk. That will generate some output that looks like:
```
{'key': 'ba1b323912a8468b90a69811c937afcf', 'status': 'ok', 'action': 'added'}
```
Finally, we make a request to Tyk that is proxied to the service. The final output from the cell will be a response from the Jokes service. That should look like the following:
```
{'value': 'Hey, funny thing... this HTTP stuff does work...'}
```

In [66]:
client, config = kubernetes.client, kubernetes.config
config.load_kube_config()
k8s = client.CoreV1Api()

create_k8s_namespace(client, k8s, service_namespace)
build_api_image()
os.system(f'kubectl -n {service_namespace} apply -f app-deploy.yaml')
define_api('openapi-v1.json', session)

# get proxied joke
requests.get('http://localhost:8080/tell/joke').json()

service/http-jokes unchanged
deployment.apps/http-jokes-service unchanged
Handling connection for 8080
{'key': 'b34132a4e39b4b6fb5a95e5f7058811d', 'status': 'ok', 'action': 'added'}
Handling connection for 8080


{'value': 'Hey, funny thing... this HTTP stuff does work...'}

The last cell just cleans things up. If you also want to stop Tyk, uncomment the last two lines.

In [62]:
kill_namespace(service_namespace)
cleanup_apis(session)
port_fwd.kill()
# os.system(f'helm uninstall -n {tyk_namespace} tyk-oss')
# kill_namespace(tyk_namespace)

cd467cdb104349368e709b5003ba6580
{"key":"cd467cdb104349368e709b5003ba6580","status":"ok","action":"deleted"}

